<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Complaints Analysis Integration with Customer360 using VantageCloud Lake on Google and Gemini
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style='font-size:16px;font-family:Arial;color:#00233C'>Complaints Analysis Integration with Customer360 is a comprehensive approach to managing customer complaints and feedback within the framework of a Customer 360-degree view using <b>VantageCloud Lake</b> on Google and <b>Google Gemini LLM model</b>. This integration aims to provide a seamless and personalized customer experience by leveraging data from various sources, including CRM systems, marketing platforms, and social media.</p> <p style='font-size:16px;font-family:Arial;color:#00233C'>The key components of this integration include:</p> 

<ol style='font-size:16px;font-family:Arial;color:#00233C'> <li><b>Customer 360 Data Manager</b>: Responsible for managing and maintaining a comprehensive view of customer data, including collection, integration, and analysis from multiple sources.</li> <li><b>Complaints Management Dashboard</b>: Analyzes customer complaints, providing insights into complaint volume, trends, and resolution progress.</li> <li><b>Customer Insights</b>: Tools for gaining insights into customer behavior and preferences, enabling targeted marketing campaigns and informed business decisions.</li> </ol> <p style='font-size:16px;font-family:Arial;color:#00233C'>The benefits of this integration include:</p> <ol style='font-size:16px;font-family:Arial;color:#00233C'> <li><b>Improved Customer Experience</b>: By integrating complaints analysis with Customer 360, businesses can address customer complaints more effectively, leading to increased customer satisfaction and loyalty.</li> <li><b>Data-Driven Decision Making</b>: The integration provides a centralized platform for analyzing customer data, enabling businesses to make informed decisions about product development, marketing strategies, and customer engagement.</li> <li><b>Enhanced Customer Insights</b>: The comprehensive view of customer data allows businesses to better understand customer needs and preferences, leading to more targeted and effective marketing efforts.</li> </ol> 


<p style='font-size:16px;font-family:Arial;color:#00233C'>By integrating complaints analysis with Customer 360, businesses can create a more comprehensive and personalized customer experience, driving business growth and customer satisfaction.</p> 

<p style = 'font-size:16px;font-family:Arial;color:#00233c'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Create a connection to VantageCloud Lake using teradataml</li>
    <li>Configuring the VantageCloud Lake Environment</li>
    <li>Getting Data for This Demo</li>
    <li>Using LLM for Sentiment Analysis, Topic Modelling and Complaint Summarization</li>
    <li>Integrated data with customer 360</li>
    <li>Cleanup</li>
</ol>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Create a connection to VantageCloud Lake using teradataml</b>


<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Import the required libraries</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [4]:
# Import the Python library teradataml and the specific environment setup modules.
from teradataml import create_context, remove_context, set_config_params
from teradataml import create_env, remove_env, get_env, list_base_envs, list_user_envs
from teradataml import DataFrame, copy_to_sql, Apply, configure, execute_sql
from teradatasqlalchemy.types import BIGINT, VARCHAR, INTEGER, FLOAT, CHAR
from teradataml.options.display import display
from teradataml import set_auth_token

# genai
from teradatagenai.convAI import tdmlAI
from teradatagenai.text_analytics import TextAnalyticsAI

import getpass
import pandas as pd
import json, os, sys, warnings

warnings.filterwarnings("ignore")
display.print_sqlmr_query = True
display.suppress_vantage_runtime_warnings = True

# for plots
import plotly.express as px

display.max_rows = 5
pd.set_option("display.max_colwidth", None)

In [ ]:
# Create the connection
print("Creating the context...")

host = getpass.getpass("Host: ")
username = getpass.getpass("Username: ")
password = getpass.getpass("Password: ")

con = create_context(host=host, username=username, password=password)

Creating the context...
Host: ········
Username: ········
Password: ········


In [ ]:
# Set the Authentication token to connect to User Environment Service (UES) in VantageCloud Lake.
ues_url = r"https://vmo.staging.innovationlabs.teradata.com/api/accounts/c1685411-eb18-4a19-8a02-aa0f20d7dff1/open-analytics"
set_auth_token(ues_url=ues_url)

Opening the browser with URL - https://vmo.staging.innovationlabs.teradata.com/auth/as/user_authz.oauth2?user_code=6P8M-7Y8G

Opened a session in browser. Authenticate yourself in the browser. In case if not able to access the opened session in browser, Open a session in any browser on any machine using URL - https://vmo.staging.innovationlabs.teradata.com/auth/as/user_authz.oauth2?user_code=6P8M-7Y8G


True

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Configuring the VantageCloud Lake Environment</b>

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>2.1 Create a new user environment</b></p>

In [ ]:
# Use Environment setup
# Create a new user environment
# create a new python user environment for python 3.10.
demo_env = create_env(
    env_name="demo_env", base_env="python_3.10", desc="Demo env for LLM"
)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Getting Data for This Demo</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_cloud');"        # Takes 1 minute
%run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_local');"        # Takes 2 minutes

In [10]:
customer_data = DataFrame(in_schema("DEMO_ComplaintAnalysis", "Customer_360_Details"))
customer_data

Customer Identifier,Name,City,State,Customer Type,Product Holdings,Total Deposit Balance,Total Credit Balance,Total Investments AUM,Customer Profitability,Customer Lifetime Value,Bank Tenure,Affluence Segment,Digital Banking Segment,Branch Banking Segment
456789123,Alphonse S. Mercado,West Covina,CA,Consumer & Business,"Checking, Home Equity, Brokerage",23291,122422,68369,312,3712,10.2,Private Banking,Not Online,Branch only
123456789,John Q. Public,Rancho Bernardo,CA,Consumer Only,"Checking, Savings, Credit Card",1425,5241,0,105,425,4.5,Mass Affluent,Digital Monetary,Branch & ATM
789456123,Linda H. Gomes,Fife,WA,Consumer Only,Mortgage,0,272162,0,225,1865,3.8,Mass Market,Not Online,None
987654321,Jane Z. Doe,El Cerrito,CA,Consumer Only,"Checking, Savings",825,0,0,42,107,6.3,Mass Market Digital Access,ATM only,None
135724681,Cardi B. Jones,Syosset,NY,Consumer Only,Credit Card,0,697,0,33,89,1.4,Mass Market,Digital Access,None


In [11]:
complaints_data = DataFrame(
    in_schema("DEMO_ComplaintAnalysis", "Customer_360_Complaints")
)


complaints_data

date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id,Customer_ID
19/12/12,Mortgage,Home equity loan or line of credit (HELOC),Trouble during payment process,None,"I had a home equity loan with Discover Bank for {$75000.00}. In both the Loan Estimate and Closing Disclosure, it states under "" Prepayment Penalty '' that there was none. I was also told this verbally. However, in a later set of documents it does describe a prepayment penalty if paid off within 36 months. I believe this discrepancy between the documents is deceptive. I believe I was lead to believe that there would not be a prepayment penalty and was surprised to find this {$420.00} penalty in my payoff letter. I spoke with a representative who agreed that it was misleading then a supervisor who noted that it was in another document which I signed. I told him I felt deceived but he was not willing to refund the prepayment penalty. I'm curious as to how many other people they are doing this to.",None,DISCOVER BANK,IL,60302,None,Consent provided,Web,19/12/12,Closed with monetary relief,Yes,None,3466832,456789123
23/09/04,Mortgage,Home equity loan or line of credit (HELOC),Applying for a mortgage or refinancing an existing mortgage,Application denials,I went to apply for XXXX home equity loan on XX/XX/XXXX. I got full pre-approval under the terms of 30 year loan interest of 9.99 %. I accepted this. They then requested lots of different information that I submitted. Every file said accepted. On XX/XX/XXXX I called them up because there was a small problem with the mortage statement I submitted. At this time I was told I was no longer approved because of debt to income ratio. The thing being I already had pre-approval and even the proof of income was already accepted. This loan would have been XXXX of my current debt payments without the loan.,None,DISCOVER BANK,WY,82801,None,Consent provided,Web,23/09/04,Closed with explanation,Yes,None,7494116,135724681
21/05/11,Mortgage,Conventional home mortgage,Closing on a mortgage,None,"Mortgage was paid off on XX/XX/2021. Home was sold, we moved to Florida. We are nearly 60 days and the escrow refund has not been issued.",None,DISCOVER BANK,FL,328XX,None,Consent provided,Web,21/05/11,Closed with monetary relief,Yes,None,4368877,789456123
19/12/02,Mortgage,Home equity loan or line of credit (HELOC),Trouble during payment process,None,"Hello, I had a second mortgage with Discover and sold the property. The second had a pre-payment penalty before making 12 payments starting XX/XX/XXXX. The property was sold in XX/XX/XXXX. I recieved a mortgage statement for XX/XX/XXXX that clearly states no pre-payment penalty on the statement. I was charged a {$480.00} pre-payment penalty. I called discover and they told me that if I was refinancing I would not have a penalty. But selling the property I do have penalty.??? My paperwork says no pre-payment penalty after 12 months of payments. Which I did make. This is rubbish trying to pass the buck based on tricky language. Either you have a pre-payment penalty or you do not. I did as my paperwork said, and reflected on my most recent statement. Please send payment to me accordingly.",None,DISCOVER BANK,FL,XXXXX,None,Consent provided,Web,19/12/02,Closed with explanation,Yes,None,3455874,987654321
22/12/04,Mortgage,Home equity loan or line of credit (HELOC),Applying for a mortgage or refinancing an existing mortgage,None,"I received a postcard from XXXX about a Home Equity Loan. While I was entering information a message came up they were running my credit. I never checked a box to authorize I wanted to complete the application nor have my credit run. I was not applying I was only trying to get information. I tried to stop by pushing the back button and then I got a message from XXXX XXX

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Using LLM for Sentiment Analysis, Topic Modelling and Complaint Summarization</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Sentiment Analysis, Topic Modelling and Complaint Summarization using Large Language Models (LLMs) revolutionizes the way we understand and categorize vast collections of text data. LLMs, like from Google Gemini family i.e. <b>gemini-1.5-pro</b>,<b>gemini-1.5-flash</b>,<b>gemini-1.0-pro</b>, etc. excel in understanding the semantics and context of words, enabling sophisticated topic modeling techniques.</p>

<p style='font-size:16px;font-family:Arial;color:#00233C'>Sentiment Analysis Using Large Language Models (LLMs) is a cutting-edge approach to understanding customer opinions and emotions expressed through text-based data. This advanced technique leverages the capabilities of LLMs to accurately identify and categorize sentiment as positive, negative, or neutral, providing businesses with valuable insights into customer perceptions and preferences.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>LLMs can generate coherent topics without needing predefined categories, making them ideal for exploratory analysis of diverse datasets. Moreover, their ability to capture subtle nuances in language allows for more precise topic identification, even in noisy or ambiguous texts.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Reasoning with a Chain of Thought</b>: Imagine you're trying to solve a problem. With a large language model, you start with an initial idea or question. Then, you use the model's capabilities to explore related concepts, gradually connecting them together. Each step builds upon the previous one, leading you closer to understanding or solving the problem. It's like putting together puzzle pieces, one by one, until you see the whole picture.</p>

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>4.1 API set-up for text analytics</b></p>

In [ ]:
cred = {
    "key": getpass(),
    "region": "us-central1-a",
    "project": "csae",
    "model_id": "gemini-1.5-pro-latest",
    "model_kwargs": {
        "temperature": 0.9,
        "top_p": 1.0,
        "top_k": 32,
        "candidate_count": 1,
        "max_output_tokens": 8192,
    },
}



llm = tdmlAI(cred=cred, vertex_ai=True)

In [ ]:
TA_obj = TextAnalyticsAI(llm=llm)

In [ ]:
# sentiment
prompt = f"""User prompt: 
        The following is text from a review:

        {consumer_complaint_narrative}

        Categorize the review as one of the following:

        Positive
        Negative
        Neutral"""

df_pred_sent = TA_obj.analyze_sentiment(
    column=tdf.consumer_complaint_narrative,
    prompt=prompt,
)

df_pred_sent = df_pred_sent.to_pandas()

In [ ]:
# topic
prompt = """The following is text from a complaint:

        {consumer_complaint_narrative}

        Give me reasoning as well as topic for this complaint
        Instructions for Reasoning:
        - Give me Reasoning in detail
        - Only one sentence reasoning
        Instructions for Topic:
        - The complaint falls into only one of the following Topics: Mortgage Application, Payment Trouble, Mortgage Closing, Report Inaccuracy, Payment Struggle
        - Only select one Topic

        My output comes in the format:
        Topic: ,    
        Reasoning:"""

df_pred_topic = TA_obj.classify(
    column=tdf.consumer_complaint_narrative,
    topics=[
        "Mortgage Application",
        "Payment Trouble",
        "Mortgage Closing",
        "Report Inaccuracy",
        "Payment Struggle",
    ],
    prompt=prompt,
)

df_pred_topic = df_pred_topic.to_pandas()

In [ ]:
# summary
prompt = prompt = f'''
            The following is text from a Bank Review:

            {consumer_complaint_narrative}

            Give me reasoning as well as summary for this review.

            Instructions for Reasoning:
            - Give me Reasoning in short
            - Only one sentence reasoning
            Instructions for Summary:
            - A short one sentence Summary of everything the review states.

            My output comes in the format:
            Summary: ,
            Reasoning:'''

df_pred_summary = TA_obj.summarize(
    column=tdf.consumer_complaint_narrative,
    prompt=prompt,
)

df_pred_summary = df_pred_summary.to_pandas()

In [ ]:
import functools
dfs = [df_pred_sent, df_pred_topic, df_pred_summary]
pd_df = functools.reduce(lambda left, right: pd.merge(left, right, on='name'), dfs)

pd_df["Sentiment"] = pd_df["Sentiment"].apply(lambda x: x.strip())
pd_df["Topic"] = pd_df["Topic"].apply(lambda x: x.strip())
pd_df["Summary"] = pd_df["Summary"].apply(lambda x: x.strip())

combined_df = customer_data.to_pandas().join(pd_df)

In [ ]:
combined_df["Bank Strategy"] = [
    "Wealth Manager to contact customer immediately",
    "Send Policy Letter from Mortgage Servicing",
    "Send Policy Letter from Executive Office",
    "Mortgage Banker to follow-up with Title Company for documentation and contact customer",
    "Branch Manager to contact customer immediately",
]

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Integrated data with customer 360</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following is an example of the output from LLM integrated with existing customer360 data. Please scroll to the right to see all the columns.</p>

In [19]:
pd.set_option("display.max_colwidth", None)
combined_df

,Customer Identifier,Name,City,State,Customer Type,Product Holdings,Total Deposit Balance,Total Credit Balance,Total Investments AUM,Customer Profitability,...,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id,Customer_ID,Sentiment,Topic,Summary,Bank Strategy
0,456789123,Alphonse S. Mercado,West Covina,CA,Consumer & Business,"Checking, Home Equity, Brokerage",23291,122422,68369,312,...,2022-12-04,Closed with non-monetary relief,Yes,None,6279246,123456789,Negative,Mortgage Application,"A customer was trying to get information about a home equity loan, but had their credit run without their consent, and is requesting the credit inquiry be removed from their credit.",Wealth Manager to contact customer immediately
1,987654321,Jane Z. Doe,El Cerrito,CA,Consumer Only,"Checking, Savings",825,0,0,42,...,2019-12-02,Closed with explanation,Yes,None,3455874,987654321,Positive,Payment Trouble,"The reviewer had a second mortgage with Discover and sold the property, but was charges a pre-payment penalty despite having make no pre-payments after 12 months of payments.",Send Policy Letter from Mortgage Servicing
2,135724681,Cardi B. Jones,Syosset,NY,Consumer Only,Credit Card,0,697,0,33,...,2019-12-12,Closed with monetary relief,Yes,None,3466832,456789123,Negative,Payment Trouble,"The reviewer had a home equity loan with Discover Bank, and was surprised to find that there was a prepayment penalty in the payoff letter, despite it not being mentioned in the Loan Estimate or Closing Disclosure.",Send Policy Letter from Executive Office
3,789456123,Linda H. Gomes,Fife,WA,Consumer Only,Mortgage,0,272162,0,225,...,2021-05-11,Closed with monetary relief,Yes,None,4368877,789456123,Negative,Payment Trouble,"The Bank Review states that the mortgage was paid off on XX/XX/2021 and that the customer moved to Florida, but the escrow refund has not been issued.",Mortgage Banker to follow-up with Title Company for documentation and contact customer
4,123456789,John Q. Public,Rancho Bernardo,CA,Consumer Only,"Checking, Savings, Credit Card",1425,5241,0,105,...,2023-09-04,Closed with explanation,Yes,None,7494116,135724681,Neutral,Mortgage Application,"The reviewer applied for an XXXX home equity loan, was pre-approved with a 30 year loan interest of 9.99%, but was later told they were no longer",Branch Manager to contact customer immediately


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Save the results back to Vantage.</p>

In [ ]:
copy_to_sql(df=combined_df, table_name="Customer_360_Complaints", if_exists="replace")

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>6. Cleanup</b>

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial; color: #00233C;'>The dataset is sourced from <a href='https://www.consumerfinance.gov/data-research/consumer-complaints/'>Consumer Financial Protection Bureau</a></p>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved
        </div>
    </div>
</footer>